# Frequentist Inference Case Study - Part B

## Learning objectives

Welcome to Part B of the Frequentist inference case study! The purpose of this case study is to help you apply the concepts associated with Frequentist inference in Python. In particular, you'll practice writing Python code to apply the following statistical concepts: 
* the _z_-statistic
* the _t_-statistic
* the difference and relationship between the two
* the Central Limit Theorem, including its assumptions and consequences
* how to estimate the population mean and standard deviation from a sample
* the concept of a sampling distribution of a test statistic, particularly for the mean
* how to combine these concepts to calculate a confidence interval

In the previous notebook, we used only data from a known normal distribution. **You'll now tackle real data, rather than simulated data, and answer some relevant real-world business problems using the data.**

## Hospital medical charges

Imagine that a hospital has hired you as their data scientist. An administrator is working on the hospital's business operations plan and needs you to help them answer some business questions. 

In this assignment notebook, you're going to use frequentist statistical inference on a data sample to answer the questions:
* has the hospital's revenue stream fallen below a key threshold?
* are patients with insurance really charged different amounts than those without?

Answering that last question with a frequentist approach makes some assumptions, and requires some knowledge, about the two groups.

We are going to use some data on medical charges obtained from [Kaggle](https://www.kaggle.com/easonlai/sample-insurance-claim-prediction-dataset). 

For the purposes of this exercise, assume the observations are the result of random sampling from our single hospital. Recall that in the previous assignment, we introduced the Central Limit Theorem (CLT), and its consequence that the distributions of sample statistics approach a normal distribution as $n$ increases. The amazing thing about this is that it applies to the sampling distributions of statistics that have been calculated from even highly non-normal distributions of data! Recall, also, that hypothesis testing is very much based on making inferences about such sample statistics. You're going to rely heavily on the CLT to apply frequentist (parametric) tests to answer the questions in this notebook.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import t
from scipy import stats
from numpy.random import seed
medical = pd.read_csv('insurance2.csv')

In [2]:
medical.shape

(1338, 8)

In [3]:
medical.head()

,age,sex,bmi,children,smoker,region,charges,insuranceclaim
0,19,0,27.900,0,1,3,16884.92400,1
1,18,1,33.770,1,0,2,1725.55230,1
2,28,1,33.000,3,0,2,4449.46200,0
3,33,1,22.705,0,0,1,21984.47061,0
4,32,1,28.880,0,0,1,3866.85520,1


__Q1:__ Plot the histogram of charges and calculate the mean and standard deviation. Comment on the appropriateness of these statistics for the data.

__A:__

In [4]:
# The mean appears to $13,270
m = np.mean(medical['charges'])

In [5]:
# Each standard deviation is $12,105
s = np.std(medical['charges'])

__Q2:__ The administrator is concerned that the actual average charge has fallen below 12,000, threatening the hospital's operational model. On the assumption that these data represent a random sample of charges, how would you justify that these data allow you to answer that question? And what would be the most appropriate frequentist test, of the ones discussed so far, to apply?

__A:__ I would try to justify that the mean is close to $12,000 because a Z-test with a 95% confidence would give us a confidence interval that will be around our sample mean. 

__Q3:__ Given the nature of the administrator's concern, what is the appropriate confidence interval in this case? A ***one-sided*** or ***two-sided*** interval? (Refresh your understanding of this concept on p. 399 of the *AoS*). Calculate the critical value and the relevant 95% confidence interval for the mean, and comment on whether the administrator should be concerned.

__A:__ A two tailed test will help us obtain a smaller confidence interval so using this approach would be better.

In [6]:
# The z-table gives us a critical value of 1.96
cv = 1.96
standard_error = s / np.sqrt(len(medical))
margin_error = cv * standard_error
# It says that our mean will be off from the population mean by $648
margin_error

648.6496792559246

In [7]:
lower_limit = m - margin_error
upper_limit = m + margin_error
print('The confidence interval using the z score is: [{},{}]'.format(lower_limit, upper_limit))

The confidence interval using the z score is: [12621.772585885332,13919.071944397181]


The confidence interval tells us that the mean is between 12621  and  13919, so the administrator should not be concerned.

The administrator then wants to know whether people with insurance really are charged a different amount to those without.

__Q4:__ State the null and alternative hypothesis here. Use the _t_-test for the difference between means, where the pooled standard deviation of the two groups is given by:
\begin{equation}
s_p = \sqrt{\frac{(n_0 - 1)s^2_0 + (n_1 - 1)s^2_1}{n_0 + n_1 - 2}}
\end{equation}

and the *t*-test statistic is then given by:

\begin{equation}
t = \frac{\bar{x}_0 - \bar{x}_1}{s_p \sqrt{1/n_0 + 1/n_1}}.
\end{equation}

(If you need some reminding of the general definition of ***t-statistic***, check out the definition on p. 404 of *AoS*). 

What assumption about the variances of the two groups are we making here?

__A:__ Null Hypothesis: The people without insurance do not get more than those with it.

Alternative Hypothesis: The people without insurance get charged more than those with it.

Assumptions made about the variance is that the data of both these samples are close to or are equal to each other.
Meaning that the spread of their data are similar and that both samples are picked at random.

__Q5:__ Perform this hypothesis test both manually, using the above formulae, and then using the appropriate function from [scipy.stats](https://docs.scipy.org/doc/scipy/reference/stats.html#statistical-tests) (hint, you're looking for a function to perform a _t_-test on two independent samples). For the manual approach, calculate the value of the test statistic and then its probability (the p-value). Verify you get the same results from both.

__A:__ 

In [8]:
with_insurance = medical['charges'].loc[medical['insuranceclaim'] == 1]
without_insurance = medical['charges'].loc[medical['insuranceclaim'] == 0]

mean_0 = np.mean(with_insurance)
n_0 = len(with_insurance)
s_0 = np.std(with_insurance)


mean_1 = np.mean(without_insurance)
n_1 = len(without_insurance)
s_1 = np.std(without_insurance)

numerator = ((n_0 - 1) * (s_0 ** 2)) + ((n_1 - 1) * (s_1 ** 2))
sp = numerator / (n_0 + n_1 - 2)
sp = np.sqrt(sp)

sp

11512.282899205744

In [9]:
t_stats = (mean_0 - mean_1) / (sp * np.sqrt((1 / n_0) + (1 / n_1)))
t_stats

11.901306943555385

In [13]:
p_value = t.sf(np.abs(t_stats), len(with_insurance))*2  # two-sided pvalue = Prob(abs(t)>tt)
p_value

3.8772983560180834e-30

In [11]:
t_val = stats.ttest_ind_from_stats(mean_0, s_0, n_0, mean_1, s_1, n_1)
t_val

Ttest_indResult(statistic=11.901306943555387, pvalue=4.090550214373679e-31)

Congratulations! Hopefully you got the exact same numerical results. This shows that you correctly calculated the numbers by hand. Secondly, you used the correct function and saw that it's much easier to use. All you need to do is pass your data to it.

__Q6:__ Conceptual question: look through the documentation for statistical test functions in scipy.stats. You'll see the above _t_-test for a sample, but can you see an equivalent one for performing a *z*-test from a sample? Comment on your answer.

In [ ]:
print(stats.__doc__)

__A:__ There is a method called zscore that gives the zscore of a sample.

## Learning outcomes

Having completed this project notebook, you now have good hands-on experience:
* using the central limit theorem to help you apply frequentist techniques to answer questions that pertain to very non-normally distributed data from the real world
* performing inference using such data to answer business questions
* forming a hypothesis and framing the null and alternative hypotheses
* testing this using a _t_-test